# Создаём точку входа в Spark

In [2]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 50.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=bb174e049cca39367c9c2f713457db60ea03f080375ac9d2f358da60d035ffad
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [4]:
spark

# Самостоятельная работа

Требуется выяснить:
1. Какое соотношение сторон экрана телефона самое популярное,
2. Плотность пикселей у экрана.

Можно использовать только rdd.

Всего 8 баллов. 


## Считывание данных
Данные взяты отсюда: https://www.kaggle.com/iabhishekofficial/mobile-price-classification
Скачиваем и копируем в папку с нотебуком

Внутри содержится следующая информация:

* id: ID
* battery_power: Total energy a battery can store in one time (mAh)
* blue: Support bluetooth or not
* clock_speed: Speed at which microprocessor executes instructions
* dual_sim: Support dual sim or not
* fc: Front Camera mega pixels
* four_g: Support 4G or not
* int_memory: Internal Memory (GB)
* m_dep: Mobile Depth (cm)
* mobile_wt: Weight of mobile phone
* n_cores: Number of cores of processor
* pc: Primary Camera mega pixels
* px_height: Pixel Resolution Height
* px_width: Pixel Resolution Width
* ram: Random Access Memory (MB)
* sc_h: Screen Height of mobile (cm)
* sc_w: Screen Width of mobile (cm)
* talk_time: Time that a single battery charge will last
* three_g: Support 3G or not
* touch_screen: Has touch screen or not
* wifi: Support wifi or not

In [5]:
train = sc.textFile('/content/train.csv')
test = sc.textFile('/content/test.csv')

In [6]:
# Выведите содержимое каждого файла (часть строк) (1 балл)
###############

train.take(5)
###############

['battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range',
 '842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1',
 '1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2',
 '563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2',
 '615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2']

In [7]:
test.take(5)

['id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi',
 '1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0',
 '2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0',
 '3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1',
 '4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0']

##  Преобразуем train и test

In [8]:
train_first_row = train.first()

train = train\
    .filter(lambda row: row != train_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')[:-1]])

In [9]:
# Преобразуйте test (1 балл)
###############

test_first_row = test.first()
test = test\
    .filter(lambda row: row != test_first_row)\
    .map(lambda row: [float(el) for el in row.split(',')[1:]])
###############

## Объединим train и test
Найти нужную функцию можно [здесь](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.html#rdd-apis)

PS: нужно сделать средсвтвами rdd pd.concat([train, test,], axis=0)

In [10]:
# Объедините train и test (2 балла)
###############
# Ваш код здесь

data = train.union(test)
data.count()
###############

3000

In [11]:
train.count()

2000

In [12]:
test.count()

1000

## Рассчитайте соотношение сторон телефона и экрана

In [13]:
data_header = train_first_row.split(',')[:-1]
data_header

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi']

In [14]:
px_height = data_header.index('px_height')
px_width = data_header.index('px_width')

In [15]:
# Выведите отсортированное распределение соотношений сторон экрана(1 балла)
# в разрезе широкоформатные или нет (экран широкоформатный, если соотногшение >=16:9)
###############

data\
    .map(lambda row: (row[px_width], row[px_height]))\
    .filter(lambda row: row[1]!= 0)\
    .map(lambda row: (('high_resolution' if row[0]/row[1] >= 1.77 else 'low_resolution'),1))\
    .reduceByKey(lambda x,y: x + y)\
    .sortByKey()\
    .collect()

[('high_resolution', 1643), ('low_resolution', 1353)]

In [16]:
import math

In [17]:
sc_h = data_header.index('sc_h')
sc_w = data_header.index('sc_w')

In [18]:
# Выведите отсортированное распределение плотности пикселей (1 балла)
# точек на дюйм
###############
data\
    .map(lambda row: (round((math.sqrt(row[px_width]**2 + row[px_height]**2) ) / (math.sqrt(row[sc_h]**2 + row[sc_w]**2))), 1))\
    .reduceByKey(lambda x,y: x + y)\
    .sortByKey(True)\
    .collect()
###############

[(22, 1),
 (24, 3),
 (25, 2),
 (26, 3),
 (27, 3),
 (28, 3),
 (29, 5),
 (30, 3),
 (31, 11),
 (32, 11),
 (33, 17),
 (34, 13),
 (35, 13),
 (36, 11),
 (37, 9),
 (38, 15),
 (39, 15),
 (40, 17),
 (41, 11),
 (42, 9),
 (43, 16),
 (44, 15),
 (45, 14),
 (46, 20),
 (47, 21),
 (48, 27),
 (49, 17),
 (50, 23),
 (51, 9),
 (52, 21),
 (53, 16),
 (54, 31),
 (55, 22),
 (56, 16),
 (57, 23),
 (58, 15),
 (59, 23),
 (60, 21),
 (61, 20),
 (62, 23),
 (63, 13),
 (64, 19),
 (65, 26),
 (66, 21),
 (67, 29),
 (68, 23),
 (69, 30),
 (70, 14),
 (71, 29),
 (72, 23),
 (73, 22),
 (74, 26),
 (75, 21),
 (76, 20),
 (77, 25),
 (78, 29),
 (79, 27),
 (80, 25),
 (81, 26),
 (82, 34),
 (83, 16),
 (84, 25),
 (85, 14),
 (86, 19),
 (87, 25),
 (88, 20),
 (89, 33),
 (90, 24),
 (91, 21),
 (92, 16),
 (93, 22),
 (94, 21),
 (95, 22),
 (96, 17),
 (97, 24),
 (98, 26),
 (99, 14),
 (100, 16),
 (101, 19),
 (102, 26),
 (103, 25),
 (104, 22),
 (105, 16),
 (106, 18),
 (107, 17),
 (108, 20),
 (109, 22),
 (110, 13),
 (111, 25),
 (112, 20),
 (113, 1

# JOIN
Повторите вышеописанное задание с помощью одной из функций ниже (отдельно рассчитайте для train и test, затем объедините результат)

Пример для двух RDDs: (rdd = {(1, 2), (3, 4), (3, 6)} other = {(3, 9)})

Имя функции |	Purpose |	Example |	Result
------------- |	------- |	------- |	------
subtractByKey |Remove elements with a key present in the other RDD.| rdd.subtractByKey(other) | {(1, 2)}
join | Perform an inner join between two RDDs. | rdd.join(other) | {(3, (4, 9)), (3, (6, 9))}
rightOuterJoin | Perform a join between two RDDs where the key must be present in the first RDD. | rdd.rightOuterJoin(other) | {(3,(Some(4),9)), (3,(Some(6),9))}leftOuterJoin | Perform a join between two RDDs where the key must be present in the other RDD. | rdd.leftOuterJoin(other) | {(1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9)))}
cogroup | Group data from both RDDs sharing the same key. | rdd.cogroup(other) | {(1,([2],[])), (3,([4, 6],[9]))}


In [19]:
train_to_join = train\
    .map(lambda row: (round((math.sqrt(row[px_width]**2 + row[px_height]**2) ) / (math.sqrt(row[sc_h]**2 + row[sc_w]**2))), 1))
test_to_join = test\
    .map(lambda row: (round((math.sqrt(row[px_width]**2 + row[px_height]**2) ) / (math.sqrt(row[sc_h]**2 + row[sc_w]**2))), 1))

In [20]:
# 2 балла
###############
join_data = train_to_join.cogroup(test_to_join)
###############

In [21]:
join_data.take(2)

[(208,
  (<pyspark.resultiterable.ResultIterable at 0x7fc1f294e4d0>,
   <pyspark.resultiterable.ResultIterable at 0x7fc1f294e990>)),
 (40,
  (<pyspark.resultiterable.ResultIterable at 0x7fc1f294e450>,
   <pyspark.resultiterable.ResultIterable at 0x7fc1f294ea50>))]

In [23]:
join_data\
    .map(lambda row: (row[0], sum([len(el) for el in row[1]]) ))\
    .sortByKey(True)\
    .collect()

[(22, 1),
 (24, 3),
 (25, 2),
 (26, 3),
 (27, 3),
 (28, 3),
 (29, 5),
 (30, 3),
 (31, 11),
 (32, 11),
 (33, 17),
 (34, 13),
 (35, 13),
 (36, 11),
 (37, 9),
 (38, 15),
 (39, 15),
 (40, 17),
 (41, 11),
 (42, 9),
 (43, 16),
 (44, 15),
 (45, 14),
 (46, 20),
 (47, 21),
 (48, 27),
 (49, 17),
 (50, 23),
 (51, 9),
 (52, 21),
 (53, 16),
 (54, 31),
 (55, 22),
 (56, 16),
 (57, 23),
 (58, 15),
 (59, 23),
 (60, 21),
 (61, 20),
 (62, 23),
 (63, 13),
 (64, 19),
 (65, 26),
 (66, 21),
 (67, 29),
 (68, 23),
 (69, 30),
 (70, 14),
 (71, 29),
 (72, 23),
 (73, 22),
 (74, 26),
 (75, 21),
 (76, 20),
 (77, 25),
 (78, 29),
 (79, 27),
 (80, 25),
 (81, 26),
 (82, 34),
 (83, 16),
 (84, 25),
 (85, 14),
 (86, 19),
 (87, 25),
 (88, 20),
 (89, 33),
 (90, 24),
 (91, 21),
 (92, 16),
 (93, 22),
 (94, 21),
 (95, 22),
 (96, 17),
 (97, 24),
 (98, 26),
 (99, 14),
 (100, 16),
 (101, 19),
 (102, 26),
 (103, 25),
 (104, 22),
 (105, 16),
 (106, 18),
 (107, 17),
 (108, 20),
 (109, 22),
 (110, 13),
 (111, 25),
 (112, 20),
 (113, 1

In [27]:
train_to_join_px = train\
    .map(lambda row: (row[px_width], row[px_height]))\
    .filter(lambda row: row[1]!=0)\
    .map(lambda row: (('high_resolution' if row[0]/row[1] >= 1.77 else 'low_resolution' ), 1))

test_to_join_px = test\
    .map(lambda row: (row[px_width], row[px_height]))\
    .filter(lambda row: row[1]!=0)\
    .map(lambda row: (('high_resolution' if row[0]/row[1] >= 1.77 else 'low_resolution' ), 1))

In [28]:
join_data_px = train_to_join_px.cogroup(test_to_join_px)

In [29]:
join_data_px\
    .map(lambda row: (row[0], sum([len(el) for el in row[1]]) ))\
    .sortByKey(True)\
    .collect()

[('high_resolution', 1643), ('low_resolution', 1353)]

# DataFrame
Теперь мы знаем про Dataframe. Нужно сделать практически всё то же самое, но используя датафрейм.

In [32]:
from pyspark.sql.functions import col, round, pow, sqrt

In [30]:
# Считываем и объединяем данные (1 балл)
# Приведите все данные к правильному типу, либо считайе сразу верно (1 балл)
# Создаём колонки с соотношением сторон и плотностью пикселей (1 балл)
###############
df_train = spark.read.csv('train.csv', header=True, inferSchema=True)
df_test = spark.read.csv('test.csv', header=True, inferSchema=True)
###############

In [31]:
data = df_test.unionByName(df_train, allowMissingColumns=True)
data

DataFrame[id: int, battery_power: int, blue: int, clock_speed: double, dual_sim: int, fc: int, four_g: int, int_memory: int, m_dep: double, mobile_wt: int, n_cores: int, pc: int, px_height: int, px_width: int, ram: int, sc_h: int, sc_w: int, talk_time: int, three_g: int, touch_screen: int, wifi: int, price_range: int]

In [33]:
data = data\
    .withColumn('asc_rate', round((col('px_width') / col('px_height')), 2))\
    .withColumn('ppi', round((sqrt(pow(data.px_width, 2) + pow(data.px_height,2)) ) / (sqrt(pow(data.sc_h, 2) + pow(data.sc_w, 2)))))

In [34]:
data.show()

+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+--------+-----+
| id|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|asc_rate|  ppi|
+---+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+--------+-----+
|  1|         1043|   1|        1.8|       1| 14|     0|         5|  0.1|      193|      3| 16|      226|    1412|3476|  12|   7|        2|      0|           1|   0|       null|    6.25|103.0|
|  2|          841|   1|        0.5|       1|  4|     1|        61|  0.8|      191|      5| 12|      746|     857|3895|   6|   0|        7|      1|           0|   0|       null|    1.15|189.0|
|  3|         1807|   1|        2.8

## Сохранение


In [35]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и временем разговора в формате "1day 1hour 1minute"
# 2 балла
###############
data_save = data.select('id', 'ppi', 'talk_time')
data_save
###############

DataFrame[id: int, ppi: double, talk_time: int]

In [36]:
data_save.write.csv('train_test_result.csv', sep=',', header=True, encoding='cp1251')